#### Dependencies:

In [1]:
from KM_prediction import *
import os
CURRENT_DIR = os.getcwd()

Before using KM prediction for the first time, we have to download the ESM-1b model:

In [2]:
model, alphabet = torch.hub.load("facebookresearch/esm:v0.4.0", "esm1b_t33_650M_UR50S")

Using cache found in C:\Users\marle/.cache\torch\hub\facebookresearch_esm_v0.4.0


Example for calculating KM prediction. You can enter multiple enyzmes and substrates. Enzyme 1 will be paired with substrate 1, enzyme 2 will be paired with substrate 2 and so on...

Enter enzymes as amino acid sequences and substrate either as SMILES strings, KEGG Compound IDs, or InChI strings.

In [3]:
import pandas as pd

split = "full"

data = pd.read_pickle(join("../data", "splits", split, "val_data.pkl"))

data['substrate_IDs'] = data['substrate_IDs'].apply(list)

data['substrate_IDs'] = ['#'.join(map(str, l)) for l in data['substrate_IDs']]

data['product_IDs'] = data['product_IDs'].apply(list)

data['product_IDs'] = ['#'.join(map(str, l)) for l in data['product_IDs']]

KMvalid, KMinvalid = KM_predicton(substrate_list = data["Main Substrate"].tolist(), 
             enzyme_list = data["Sequence"].tolist(),
                                 KM = np.log10(data["Km"].tolist()),
                                 EC = data["ECs"].tolist())

Step 1/3: Calculating numerical representations for all metabolites.
.....1(a) Calculating input matrices for Graph Neural Network
.....1(b) Calculating numerical metabolite representations using a Graph Neural Network
Step 2/3: Calculating numerical representations for all enzymes.
.....2(a) Loading ESM-1b model.
.....2(b) Calculating enzyme representations.
Step 3/3: Making predictions for KM.


In [4]:
# KMvalid.to_pickle('../data/output_' + organism + '.pkl')
df = KMvalid

In [9]:
import scipy as sci
import sklearn.metrics as sk

rms = sk.mean_squared_error(df["value"].tolist(), df["KM [mM]"].tolist(), squared=False)
R2 = sk.r2_score(df["value"].tolist(), df["KM [mM]"].tolist())
Pearson = sci.stats.pearsonr(df["value"].tolist(), df["KM [mM]"].tolist())[0]

print(rms, R2, Pearson)

rms = sk.mean_squared_error((10**df["value"]).tolist(), ((10**df["KM [mM]"])/1000).tolist(), squared=False)
R2 = sk.r2_score((10**df["value"]).tolist(), ((10**df["KM [mM]"])/1000).tolist())
Pearson = sci.stats.pearsonr((10**df["value"]).tolist(), ((10**df["KM [mM]"])/1000).tolist())[0]
MAE = np.mean(abs(np.array((10**df["value"]).tolist()) - np.array(((10**df["KM [mM]"])/1000).tolist())))
MedAE = np.median(abs(np.array((10**df["value"]).tolist()) - np.array(((10**df["KM [mM]"])/1000).tolist())))

print(rms, R2, Pearson, MAE, MedAE)

2.9891739936216535 -13.19129295249482 0.5691185107453821
0.0019392709743795602 0.029436287101281988 0.314296242197024 0.0008028516654652407 0.0001407402507233201
